In [1]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
import time
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler

Initialize the data points and Random centers
===========================


In [2]:
cluster=4
np.random.seed(0)

In [3]:
features= pd.read_csv('dataset for 9.csv') 
features.label.unique()

array(['alternative', 'blues', 'electronic', 'folkcountry', 'funksoulrnb',
       'jazz', 'pop', 'raphiphop', 'rock'], dtype=object)

In [4]:
features=features.drop(features[features.label=='rock'].index)
features=features.drop(features[features.label=='raphiphop'].index)
features=features.drop(features[features.label=='pop'].index)
features=features.drop(features[features.label=='jazz'].index)
features=features.drop(features[features.label=='funksoulrnb'].index)
features.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,50_Minutes-Colours.wav,0.370957,0.191483,1554.272672,2272.273786,3586.444289,0.039205,-128.463409,117.550293,32.394691,...,4.688472,-7.322381,-0.384554,-0.297131,-0.951402,-5.560275,2.688139,-3.069870,2.455562,alternative
1,50_Minutes-Nightwalk.wav,0.301646,0.118957,892.314417,1477.003664,1649.473129,0.033195,-228.945496,162.367874,25.274275,...,-2.929274,-5.459329,-3.360488,-5.309353,-2.012183,-8.864375,-0.077231,-6.165314,0.011491,alternative
2,50_Minutes-The_Great_Wall.wav,0.375948,0.203939,1270.664430,1838.236128,2720.278635,0.043340,-114.051369,133.875641,-2.535841,...,1.340597,-1.123214,4.357597,-1.194922,-6.105379,-10.633196,-7.830831,-9.174461,1.239748,alternative
3,Abigail_Lapell-Dress_Rehearsal.wav,0.288743,0.124793,1946.785743,2323.686436,4007.160408,0.080200,-159.387787,104.808685,-9.930721,...,0.982948,-14.001296,2.717434,-12.282285,-2.516945,-7.880888,-3.486890,-5.966151,-2.050257,alternative
4,Albino_Catfish-Ice_Cream.wav,0.291857,0.154149,2477.062284,2280.878176,4829.267342,0.135053,-123.162804,69.661911,-16.024954,...,-8.382612,-10.428762,-10.952260,-3.459139,-4.728041,-5.685522,-10.800599,-2.911047,-0.371204,alternative


In [5]:
x= features.iloc[:,1:-1]
y=features.iloc[:,-1]
x=np.asarray(x)

In [6]:
scaler = StandardScaler()
x = scaler.fit_transform(np.array(x, dtype = float))

In [7]:
# from sklearn.preprocessing import MinMaxScaler 
# scaler = MinMaxScaler()
# x=scaler.fit_transform(x)

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)
print(np.unique(y))

[0 1 2 3]


In [9]:
def generateRandomCenters(c,x):
    centers=[]
    _min= (np.min(x))
    _max= (np.max(x))
    t= np.mean(x,axis=0)    
    for i in range(c):
        centers.append( t + 0.01*i)   
    return np.asarray(centers) 

generateRandomCenters(cluster,x)

array([[ 3.67113747e-16, -9.47390314e-17, -1.18423789e-17,
        -7.04621546e-16,  4.14483263e-16,  3.55271368e-17,
         9.47390314e-17,  3.31586610e-16, -1.18423789e-17,
        -2.36847579e-16, -9.76996262e-17, -2.07241631e-17,
        -1.18423789e-17,  1.65793305e-16,  1.18423789e-17,
         2.96059473e-17,  2.13162821e-16, -5.92118946e-17,
         1.06581410e-16, -1.77635684e-17,  1.45069142e-16,
         0.00000000e+00,  1.77635684e-16, -1.77635684e-17,
        -1.00660221e-16, -5.92118946e-18],
       [ 1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00

In [10]:
# intialize random member ship
def initializeMembershipMatrix(n):
    membership_mat = list()
    for i in range(n.shape[0]):
#         random_num_list = [random.random() for i in range(cluster)]
        random_num_list = [ np.random.rand() for i in range(cluster)]
        summation = sum(random_num_list) 
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list) 
    return np.asarray(membership_mat) 

In [11]:
def updateMembershipValue3_2(u, v,val):
    distances = cdist(val, v,metric='euclidean')
    return distances

In [12]:

def CalculateWeightValues(cluster_center,val,q):
        
    membership=[]    
    distances = cdist(val, v,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances)
    membership= distances/sumdistance 
    
    return membership


In [13]:

def CalculateWeightValuesByIndex(cluster_center,val,q,index):
    
#     print(centers[q])
#     distances=[]
#     for j in range(val.shape[0]):
#         distances.append(np.linalg.norm(val[j]- v[q])) 
    
    distances = cdist(val, v,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances)
    membership= distances[index]/sumdistance
        
    return membership


In [14]:
def calcError(center,mem):
    dist=0
    for i in range(mem.shape[0]):
        dist+=np.linalg.norm(center - mem[i])
    return dist

In [15]:
def moveVtoTheCenter(v,u,x):
    for k in range( cluster):
        items=[]
        for i  in range(  (x.shape[0])):
            if(u[i,k]>=np.max(u[i])):
                items.append(x[i])
        v[k]=np.mean(items, axis=0)
    return v

In [16]:
def _k_init(X, n_clusters, x_squared_norms, random_state, n_local_trials=None):
    """Init n_clusters seeds according to k-means++
    Parameters
    ----------
    X : array or sparse matrix, shape (n_samples, n_features)
        The data to pick seeds for. To avoid memory copy, the input data
        should be double precision (dtype=np.float64).
    n_clusters : integer
        The number of seeds to choose
    x_squared_norms : array, shape (n_samples,)
        Squared Euclidean norm of each data point.
    random_state : int, RandomState instance
        The generator used to initialize the centers. Use an int to make the
        randomness deterministic.
        See :term:`Glossary <random_state>`.
    n_local_trials : integer, optional
        The number of seeding trials for each center (except the first),
        of which the one reducing inertia the most is greedily chosen.
        Set to None to make the number of trials depend logarithmically
        on the number of seeds (2+log(k)); this is the default.
    Notes
    -----
    Selects initial cluster centers for k-mean clustering in a smart way
    to speed up convergence. see: Arthur, D. and Vassilvitskii, S.
    "k-means++: the advantages of careful seeding". ACM-SIAM symposium
    on Discrete algorithms. 2007
    Version ported from http://www.stanford.edu/~darthur/kMeansppTest.zip,
    which is the implementation used in the aforementioned paper.
    """
    n_samples, n_features = X.shape

    centers = np.empty((n_clusters, n_features), dtype=X.dtype)

    assert x_squared_norms is not None, 'x_squared_norms None in _k_init'

    # Set the number of local seeding trials if none is given
    if n_local_trials is None:
        # This is what Arthur/Vassilvitskii tried, but did not report
        # specific results for other than mentioning in the conclusion
        # that it helped.
        n_local_trials = 2 + int(np.log(n_clusters))

    # Pick first center randomly
    center_id = random_state.randint(n_samples)
    if sp.issparse(X):
        centers[0] = X[center_id].toarray()
    else:
        centers[0] = X[center_id]

    # Initialize list of closest distances and calculate current potential
    closest_dist_sq = euclidean_distances(
        centers[0, np.newaxis], X, Y_norm_squared=x_squared_norms,
        squared=True)
    current_pot = closest_dist_sq.sum()

    # Pick the remaining n_clusters-1 points
    for c in range(1, n_clusters):
        # Choose center candidates by sampling with probability proportional
        # to the squared distance to the closest existing center
        rand_vals = random_state.random_sample(n_local_trials) * current_pot
        candidate_ids = np.searchsorted(stable_cumsum(closest_dist_sq),
                                        rand_vals)

        # Compute distances to center candidates
        distance_to_candidates = euclidean_distances(
            X[candidate_ids], X, Y_norm_squared=x_squared_norms, squared=True)

        # Decide which candidate is the best
        best_candidate = None
        best_pot = None
        best_dist_sq = None
        for trial in range(n_local_trials):
            # Compute potential when including center candidate
            new_dist_sq = np.minimum(closest_dist_sq,
                                     distance_to_candidates[trial])
            new_pot = new_dist_sq.sum()

            # Store result if it is the best local trial so far
            if (best_candidate is None) or (new_pot < best_pot):
                best_candidate = candidate_ids[trial]
                best_pot = new_pot
                best_dist_sq = new_dist_sq

        # Permanently add best center candidate found in local tries
        if sp.issparse(X):
            centers[c] = X[best_candidate].toarray()
        else:
            centers[c] = X[best_candidate]
        current_pot = best_pot
        closest_dist_sq = best_dist_sq

    return centers

In [17]:
# x_squared_norms = np.row_norms(x, squared=True)

from sklearn.utils.extmath import row_norms
from sklearn.utils.extmath import row_norms, squared_norm, stable_cumsum
import scipy.sparse as sp
from sklearn.metrics.pairwise import euclidean_distances
 
c = _k_init(x,cluster, row_norms(x, squared=True), np.random.RandomState(seed=42))
c

array([[ 0.14591782,  2.36086056,  0.5599574 ,  0.18043443,  0.3842748 ,
         0.60186878,  1.60584676, -0.56374168, -0.72131979,  1.3689187 ,
        -0.22696825, -0.70423403, -0.67377537, -0.01129317,  0.35331333,
        -0.48716719, -0.60666531,  0.47072052, -0.04794361, -0.57929162,
        -0.543044  ,  0.3092124 , -0.26573023, -0.63712485, -1.01964212,
        -1.08770455],
       [-1.4709506 , -1.51168168, -0.41830045, -0.27758505, -0.55010588,
        -0.54060227, -1.89934683,  0.12074606,  1.10828553,  0.07370339,
         1.37246559,  0.07163308, -1.13589519, -1.46562504, -1.78573707,
        -0.89739533, -1.74849113, -2.93993077, -1.80282294, -2.21502041,
        -0.74278132, -1.53353023, -0.60379111, -2.18608643, -0.09553641,
        -0.02075801],
       [-0.6291413 ,  0.89139233, -0.20975235,  0.39161376, -0.00776291,
        -0.51411904,  0.67001608,  0.56528791,  0.88686671, -1.66448365,
        -1.11493836, -0.45816883,  0.42704356, -0.28040365, -0.37987044,
       

Initialization
=================

In [18]:
import random
# x =np.asarray( random.sample(x.tolist(), len(x.tolist())))
# print(x[:5,])

u=initializeMembershipMatrix(x)
v=c#generateRandomCenters(cluster,x)
 
w=updateMembershipValue3_2(u,v,x)


ny=np.argmin(w,axis=1)

# for i in range ( cluster):
#     plt.scatter(x[ny == i, 0], x[ny == i, 1],  s=10, label="Cluster - "+str(i))
# plt.scatter(x=np.asarray(v)[:,0],y=np.asarray(v)[:,1], color='black',s=50)
# plt.show()  

POCS -- Parallel projection
=================

In [19]:
# val=x
errorlist=[]
start=time.time()
for t in range(50):
    print(t)
    error=0
    for q in range(cluster): 
         
        val=x[ny == q,:] 
                
        if(val.shape[0]!=0):
            
            weight=CalculateWeightValues(v[q],val,q) 

            temp=0    
            for i in range(val.shape[0]):                
                temp += weight[i]*(val[i]- v[q])
            v[q]= v[q]+temp    

        error+=calcError(v[q],val)
    
#     print(t,q,error) 
    errorlist.append([error,time.time()- start])

    w=updateMembershipValue3_2(u,v,x)  
    ny=np.argmin(w,axis=1)
    
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [20]:

from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics.cluster import normalized_mutual_info_score

print(adjusted_rand_score(y,ny))
print(mean_squared_error(y,ny))
print(normalized_mutual_info_score(y,ny))


0.026329091798082085
3.09
0.04877255611916577


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [21]:
test=pd.DataFrame()
test['y']=y
test['ny']=ny
# test.head()

In [22]:
import itertools 
perm = itertools.permutations(np.unique(y)) 
_totalperm=[]
  
for i in list(perm): 
    _totalperm.append(list(i)) 

In [23]:
from sklearn import metrics as m
accuracy=np.zeros((len(_totalperm)))
values=[]

for i in range(len(_totalperm)): 
    print(i)
    newcolvals=np.zeros((test.shape[0]))
    for j in range(test.shape[0]):
        newcolvals[j]=(_totalperm[i])[test.ny[j]] 
        
    accval=m.accuracy_score(test.y,newcolvals)
    accuracy[i]=(accval)
    values.append(list(_totalperm[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [24]:
np.max(accuracy)

0.34